Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both Bachelor semestre 1 and Bachelor semestre 6. Compute how many months it took each student to go from the first to the sixth semester. Partition the data between male and female students, and compute the average -- is the difference in average statistically significant?

In [96]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247')
htmlcontent = BeautifulSoup(r.content, 'html.parser')
print(htmlcontent.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <div>
   </div>
   <title>
   </title>
   <script src="GEDPUBLICREPORTS.txt?ww_x_path=Gestac.Base.Palette_js&amp;ww_c_langue=fr" type="text/javascript">
   </script>
   <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.Style" rel="stylesheet" type="text/css">
    <link href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.StyleNavigator" rel="stylesheet" type="text/css"/>
   </link>
  </meta>
 </head>
 <body alink="#666666" bgcolor="#ffffff" link="#666666" marginheight="0" marginwidth="5" vlink="#666666">
  <div class="filtres">
   <form action="!GEDPUBLICREPORTS.filter" method="GET" name="f">
    <input name="ww_b_list" type="hidden" value="1">
     <input name="ww_i_reportmodel" type="hidden" value="133685247">
      <input name="ww_c_langue" type="hidden" value="">
       <h1 id="titre">
        Liste des étudiants inscrits par semestre
       </h1>
       <table border="0" id="format">
 

In [98]:
computersciencefield = htmlcontent.find('option', text='Informatique')
computersciencefield.get('value')

'249847'

In [99]:
year = htmlcontent.find('option', text='2007-2008')
year.get('value')

'978181'

In [112]:
yearshtml = htmlcontent.findAll('input', attrs={'name' : 'zz_x_PERIODE_ACAD'})
#if yearshtml:
for option in yearshtml[0].findAll('option'):
    years = pd.DataFrame({'year':[option.text],'code':[option.get('value')]})
years

AttributeError: 'ResultSet' object has no attribute 'findAll'

In [90]:
yearshtml

[<input name="zz_x_PERIODE_ACAD" type="hidden" value=""><select name="ww_x_PERIODE_ACAD" onchange="document.f.zz_x_PERIODE_ACAD.value=document.f.ww_x_PERIODE_ACAD.options[document.f.ww_x_PERIODE_ACAD.selectedIndex].text"><option value="null"></option><option value="355925344">2016-2017</option><option value="213638028">2015-2016</option><option value="213637922">2014-2015</option><option value="213637754">2013-2014</option><option value="123456101">2012-2013</option><option value="123455150">2011-2012</option><option value="39486325">2010-2011</option><option value="978195">2009-2010</option><option value="978187">2008-2009</option><option value="978181">2007-2008</option></select></input>]